In [2]:
# Import necessary modules
import sys
sys.path.append('../fraud_detection')  # Adjust this path based on your working directory
from utils.logging_util import setup_logger

# notebooks/eda_fraud_detection.ipynb

# Import the necessary functions from your project
# from model_training import prepare_data, train_logistic_regression , train_random_forest , train_gradient_boosting ,train_decision_tree,  train_mlp,train_rnn,train_cnn,train_lstm
from model_training import (prepare_data, prepare_credit_card_data,train_logistic_regression, train_random_forest, 
                            train_gradient_boosting, train_decision_tree,prepare_large_data )

from model_evaluation import evaluate_model

from model_explainability import explain_model_shap, explain_model_lime


c:\Users\Yonas\Desktop\proj\yonasKu-fraud-detection-for-e-commerce-and-bank-transactions\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# data_path = '../data/fraud_datas.csv'
# prepared_data = prepare_data(data_path, target_column='fraud_class')


In [4]:
# Path to the credit card dataset
data_path = "../data/credit_datas.csv"
target_column = "Class"  # Target column indicating fraud (1 = Fraud, 0 = Non-Fraud)

# Prepare the data for credit card fraud detection
credit_data = prepare_credit_card_data(data_path, target_column)

2024-11-25 20:19:23,439 - INFO - Loaded data from ../data/credit_datas.csv
2024-11-25 20:19:23,439 - model_training - INFO - Loaded data from ../data/credit_datas.csv
2024-11-25 20:19:23,439 - model_training - INFO - Loaded data from ../data/credit_datas.csv
2024-11-25 20:19:23,439 - model_training - INFO - Loaded data from ../data/credit_datas.csv
2024-11-25 20:19:23,968 - INFO - Data successfully split into training and testing sets.
2024-11-25 20:19:23,968 - model_training - INFO - Data successfully split into training and testing sets.
2024-11-25 20:19:23,968 - model_training - INFO - Data successfully split into training and testing sets.
2024-11-25 20:19:23,968 - model_training - INFO - Data successfully split into training and testing sets.
2024-11-25 20:19:24,301 - INFO - Feature scaling applied.
2024-11-25 20:19:24,301 - model_training - INFO - Feature scaling applied.
2024-11-25 20:19:24,301 - model_training - INFO - Feature scaling applied.
2024-11-25 20:19:24,301 - model_tr

In [5]:
data_path = '../data/Fraudlent_Datas.csv'
prepared_data = prepare_large_data(data_path, target_column='class')

2024-11-25 20:19:25,671 - INFO - Loaded data from ../data/Fraudlent_Datas.csv
2024-11-25 20:19:25,671 - model_training - INFO - Loaded data from ../data/Fraudlent_Datas.csv
2024-11-25 20:19:25,671 - model_training - INFO - Loaded data from ../data/Fraudlent_Datas.csv
2024-11-25 20:19:25,671 - model_training - INFO - Loaded data from ../data/Fraudlent_Datas.csv
2024-11-25 20:19:26,624 - INFO - Calculated 'time_to_purchase' and dropped 'signup_time' and 'purchase_time'.
2024-11-25 20:19:26,624 - model_training - INFO - Calculated 'time_to_purchase' and dropped 'signup_time' and 'purchase_time'.
2024-11-25 20:19:26,624 - model_training - INFO - Calculated 'time_to_purchase' and dropped 'signup_time' and 'purchase_time'.
2024-11-25 20:19:26,624 - model_training - INFO - Calculated 'time_to_purchase' and dropped 'signup_time' and 'purchase_time'.
2024-11-25 20:19:26,733 - INFO - Encoded 'country' using frequency encoding.
2024-11-25 20:19:26,733 - model_training - INFO - Encoded 'country' u

In [6]:


# # Accessing the training and testing sets for credit card fraud detection
# X_train_credit = credit_data['X_train']
# X_test_credit = credit_data['X_test']
# y_train_credit = credit_data['y_train']
# y_test_credit = credit_data['y_test']
# scaler_credit = prepared_data.get('scaler')
# # Display confirmation
# print("Credit card fraud data successfully prepared.")
# print(f"Training set size: {X_train_credit.shape[0]} samples")
# print(f"Test set size: {X_test_credit.shape[0]} samples")


In [7]:

# # Access the prepared data
# X_train = prepared_data['X_train']
# X_test = prepared_data['X_test']
# y_train = prepared_data['y_train']
# y_test = prepared_data['y_test']
# scaler = prepared_data.get('scaler')

# # Display confirmation
# print("fraud data successfully prepared.")
# print(f"Training set size: {X_train.shape[0]} samples")
# print(f"Test set size: {X_test.shape[0]} samples")

In [8]:

# Access the prepared data
X_train = prepared_data['X_train']
X_test = prepared_data['X_test']
y_train = prepared_data['y_train']
y_test = prepared_data['y_test']
scaler = prepared_data.get('scaler')

# Display confirmation
print("Credit card fraud data successfully prepared.")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")

feature_names = prepared_data['feature_names']
print("Feature names in X_train (before scaling):")
print(feature_names)

Credit card fraud data successfully prepared.
Training set size: 103316 samples
Test set size: 25830 samples
Feature names in X_train (before scaling):
['purchase_value', 'age', 'transaction_frequency', 'transaction_count', 'transaction_velocity', 'hour_of_day', 'day_of_week', 'time_to_purchase', 'country_encoded', 'source_Direct', 'source_SEO', 'browser_FireFox', 'browser_IE', 'browser_Opera', 'browser_Safari', 'sex_M']


In [9]:

# Train and save the Logistic Regression model and its scaler
logistic_model_path = '../models/logistic_regression_model.pkl'
logistic_scaler_path = '../models/logistic_regression_scaler.pkl'
logistic_results = train_logistic_regression(X_train, X_test, y_train, y_test, logistic_model_path, scaler, logistic_scaler_path)

# Train and save the Random Forest model and its scaler
random_forest_model_path = '../models/random_forest_model.pkl'
random_forest_scaler_path = '../models/random_forest_scaler.pkl'
random_forest_results = train_random_forest(X_train, X_test, y_train, y_test, random_forest_model_path, scaler, random_forest_scaler_path)

# Display results
print(logistic_results)
print(random_forest_results)


2024-11-25 20:19:27,554 - INFO - Training model: LogisticRegression.
2024-11-25 20:19:27,554 - model_training - INFO - Training model: LogisticRegression.
2024-11-25 20:19:27,554 - model_training - INFO - Training model: LogisticRegression.
2024-11-25 20:19:27,554 - model_training - INFO - Training model: LogisticRegression.
2024-11-25 20:19:27,854 - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-25 20:19:27,854 - model_training - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-25 20:19:27,854 - model_training - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-25 20:19:27,854 - model_training - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-25 20:19:27,867 - INFO - Scaler saved to ../models/logistic_regression_scaler.pkl.
2024-11-25 20:19:27,867 - model_training - INFO - Scaler saved to ../models/logistic_regression_scaler.pkl.
2024-11-25 20:19:27,867 - model_training - INFO - Scaler saved to .

{'model': LogisticRegression(class_weight='balanced', max_iter=2000), 'training_time': 0.28107404708862305, 'roc_auc': np.float64(0.7594477170742543), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      0.64      0.76     23389\n           1       0.17      0.71      0.27      2441\n\n    accuracy                           0.64     25830\n   macro avg       0.56      0.67      0.52     25830\nweighted avg       0.88      0.64      0.72     25830\n'}
{'model': RandomForestClassifier(class_weight='balanced'), 'training_time': 57.49529266357422, 'roc_auc': np.float64(0.7720956722391218), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.98     23389\n           1       1.00      0.55      0.71      2441\n\n    accuracy                           0.96     25830\n   macro avg       0.98      0.77      0.84     25830\nweighted avg       0.96      0.96      0.95     25830\n'}


In [10]:
# Access the prepared credit card fraud data
X_train_credit = credit_data['X_train']
X_test_credit = credit_data['X_test']
y_train_credit = credit_data['y_train']
y_test_credit = credit_data['y_test']
credit_scaler = credit_data.get('scaler')

# Train and save the Logistic Regression model for credit card fraud detection
credit_logistic_model_path = '../models/credit_logistic_regression_model.pkl'
credit_logistic_scaler_path = '../models/credit_logistic_scaler.pkl'
credit_logistic_results = train_logistic_regression(
    X_train_credit, X_test_credit, y_train_credit, y_test_credit, 
    credit_logistic_model_path, credit_scaler, credit_logistic_scaler_path
)

# Train and save the Random Forest model for credit card fraud detection
credit_random_forest_model_path = '../models/credit_random_forest_model.pkl'
credit_random_forest_scaler_path = '../models/credit_random_forest_scaler.pkl'
credit_random_forest_results = train_random_forest(
    X_train_credit, X_test_credit, y_train_credit, y_test_credit, 
    credit_random_forest_model_path, credit_scaler, credit_random_forest_scaler_path
)

# Display results for credit card fraud detection models
print("\nLogistic Regression Results (Credit Card Fraud):")
print(credit_logistic_results)

print("\nRandom Forest Results (Credit Card Fraud):")
print(credit_random_forest_results)

2024-11-25 20:20:28,875 - INFO - Training model: LogisticRegression.
2024-11-25 20:20:28,875 - model_training - INFO - Training model: LogisticRegression.
2024-11-25 20:20:28,875 - model_training - INFO - Training model: LogisticRegression.
2024-11-25 20:20:28,875 - model_training - INFO - Training model: LogisticRegression.
2024-11-25 20:20:31,181 - INFO - Model saved to ../models/credit_logistic_regression_model.pkl.
2024-11-25 20:20:31,181 - model_training - INFO - Model saved to ../models/credit_logistic_regression_model.pkl.
2024-11-25 20:20:31,181 - model_training - INFO - Model saved to ../models/credit_logistic_regression_model.pkl.
2024-11-25 20:20:31,181 - model_training - INFO - Model saved to ../models/credit_logistic_regression_model.pkl.
2024-11-25 20:20:31,196 - INFO - Scaler saved to ../models/credit_logistic_scaler.pkl.
2024-11-25 20:20:31,196 - model_training - INFO - Scaler saved to ../models/credit_logistic_scaler.pkl.
2024-11-25 20:20:31,196 - model_training - INFO


Logistic Regression Results (Credit Card Fraud):
{'model': LogisticRegression(class_weight='balanced', max_iter=2000), 'training_time': 2.2915356159210205, 'roc_auc': np.float64(0.9754564584706141), 'report': '              precision    recall  f1-score   support\n\n           0       1.00      0.98      0.99     56656\n           1       0.06      0.89      0.11        90\n\n    accuracy                           0.98     56746\n   macro avg       0.53      0.93      0.55     56746\nweighted avg       1.00      0.98      0.99     56746\n'}

Random Forest Results (Credit Card Fraud):
{'model': RandomForestClassifier(class_weight='balanced'), 'training_time': 338.11589002609253, 'roc_auc': np.float64(0.9369730380620666), 'report': '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     56656\n           1       0.98      0.71      0.83        90\n\n    accuracy                           1.00     56746\n   macro avg       0.99      0.86 

In [11]:
# Define paths for Decision Tree and Gradient Boosting models and scalers
decision_tree_model_path = '../models/decision_tree_model.pkl'
decision_tree_scaler_path = '../models/decision_tree_scaler.pkl'

gradient_boosting_model_path = '../models/gradient_boosting_model.pkl'
gradient_boosting_scaler_path = '../models/gradient_boosting_scaler.pkl'


# Train and save the Decision Tree model and its scaler
decision_tree_results = train_decision_tree(
    X_train, X_test, y_train, y_test, decision_tree_model_path, scaler, decision_tree_scaler_path
)

# Train and save the Gradient Boosting model and its scaler
gradient_boosting_results = train_gradient_boosting(
    X_train, X_test, y_train, y_test, gradient_boosting_model_path, scaler, gradient_boosting_scaler_path
)

# Display results
print("\nDecision Tree Model Results:")
print(decision_tree_results)

print("\nGradient Boosting Model Results:")
print(gradient_boosting_results)


2024-11-25 20:26:11,615 - INFO - Training model: DecisionTreeClassifier.
2024-11-25 20:26:11,615 - model_training - INFO - Training model: DecisionTreeClassifier.
2024-11-25 20:26:11,615 - model_training - INFO - Training model: DecisionTreeClassifier.
2024-11-25 20:26:11,615 - model_training - INFO - Training model: DecisionTreeClassifier.
2024-11-25 20:26:14,896 - INFO - Model saved to ../models/decision_tree_model.pkl.
2024-11-25 20:26:14,896 - model_training - INFO - Model saved to ../models/decision_tree_model.pkl.
2024-11-25 20:26:14,896 - model_training - INFO - Model saved to ../models/decision_tree_model.pkl.
2024-11-25 20:26:14,896 - model_training - INFO - Model saved to ../models/decision_tree_model.pkl.
2024-11-25 20:26:14,909 - INFO - Scaler saved to ../models/decision_tree_scaler.pkl.
2024-11-25 20:26:14,909 - model_training - INFO - Scaler saved to ../models/decision_tree_scaler.pkl.
2024-11-25 20:26:14,909 - model_training - INFO - Scaler saved to ../models/decision_tr


Decision Tree Model Results:
{'model': DecisionTreeClassifier(), 'training_time': 3.2705187797546387, 'roc_auc': np.float64(0.7564399340446334), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      0.94      0.95     23389\n           1       0.49      0.58      0.53      2441\n\n    accuracy                           0.90     25830\n   macro avg       0.72      0.76      0.74     25830\nweighted avg       0.91      0.90      0.91     25830\n'}

Gradient Boosting Model Results:
{'model': GradientBoostingClassifier(), 'training_time': 49.88558793067932, 'roc_auc': np.float64(0.7809695885184598), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.98     23389\n           1       1.00      0.55      0.71      2441\n\n    accuracy                           0.96     25830\n   macro avg       0.98      0.77      0.84     25830\nweighted avg       0.96      0.96      0.95     25830\n'}


In [12]:
credit_decision_tree_model_path = '../models/credit_decision_tree_model.pkl'
credit_decision_tree_scaler_path = '../models/credit_decision_tree_scaler.pkl'

credit_gradient_boosting_model_path = '../models/credit_gradient_boosting_model.pkl'
credit_gradient_boosting_scaler_path = '../models/credit_gradient_boosting_scaler.pkl'


# Train and save the Decision Tree model for credit card fraud detection
credit_decision_tree_results = train_decision_tree(
    X_train_credit, X_test_credit, y_train_credit, y_test_credit,
    credit_decision_tree_model_path, credit_scaler, credit_decision_tree_scaler_path
)

# Train and save the Gradient Boosting model for credit card fraud detection
credit_gradient_boosting_results = train_gradient_boosting(
    X_train_credit, X_test_credit, y_train_credit, y_test_credit,
    credit_gradient_boosting_model_path, credit_scaler, credit_gradient_boosting_scaler_path
)

# Display results for credit card fraud detection models
print("\nDecision Tree Model Results (Credit Card Fraud):")
print(credit_decision_tree_results)

print("\nGradient Boosting Model Results (Credit Card Fraud):")
print(credit_gradient_boosting_results)

2024-11-25 20:27:05,137 - INFO - Training model: DecisionTreeClassifier.
2024-11-25 20:27:05,137 - model_training - INFO - Training model: DecisionTreeClassifier.
2024-11-25 20:27:05,137 - model_training - INFO - Training model: DecisionTreeClassifier.
2024-11-25 20:27:05,137 - model_training - INFO - Training model: DecisionTreeClassifier.
2024-11-25 20:28:13,743 - INFO - Model saved to ../models/credit_decision_tree_model.pkl.
2024-11-25 20:28:13,743 - model_training - INFO - Model saved to ../models/credit_decision_tree_model.pkl.
2024-11-25 20:28:13,743 - model_training - INFO - Model saved to ../models/credit_decision_tree_model.pkl.
2024-11-25 20:28:13,743 - model_training - INFO - Model saved to ../models/credit_decision_tree_model.pkl.
2024-11-25 20:28:13,756 - INFO - Scaler saved to ../models/credit_decision_tree_scaler.pkl.
2024-11-25 20:28:13,756 - model_training - INFO - Scaler saved to ../models/credit_decision_tree_scaler.pkl.
2024-11-25 20:28:13,756 - model_training - IN


Decision Tree Model Results (Credit Card Fraud):
{'model': DecisionTreeClassifier(), 'training_time': 68.59244465827942, 'roc_auc': np.float64(0.8608640057736359), 'report': '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     56656\n           1       0.70      0.72      0.71        90\n\n    accuracy                           1.00     56746\n   macro avg       0.85      0.86      0.85     56746\nweighted avg       1.00      1.00      1.00     56746\n'}

Gradient Boosting Model Results (Credit Card Fraud):
{'model': GradientBoostingClassifier(), 'training_time': 1011.3606970310211, 'roc_auc': np.float64(0.7665101666195988), 'report': '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     56656\n           1       0.90      0.63      0.75        90\n\n    accuracy                           1.00     56746\n   macro avg       0.95      0.82      0.87     56746\nweighted avg       1.00

In [13]:
# Extract the model objects from the training results
logistic_model = logistic_results["model"]
decision_tree_model = decision_tree_results["model"]
random_forest_model = random_forest_results["model"]
gradient_boosting_model = gradient_boosting_results["model"]

# Evaluate each model
logistic_evaluation = evaluate_model(logistic_model, X_test, y_test)
decision_tree_evaluation = evaluate_model(decision_tree_model, X_test, y_test)
random_forest_evaluation = evaluate_model(random_forest_model, X_test, y_test)
gradient_boosting_evaluation = evaluate_model(gradient_boosting_model, X_test, y_test)

# Display evaluation results
print("Logistic Regression Evaluation:")
print(logistic_evaluation)

print("\nDecision Tree Evaluation:")
print(decision_tree_evaluation)

print("\nRandom Forest Evaluation:")
print(random_forest_evaluation)

print("\nGradient Boosting Evaluation:")
print(gradient_boosting_evaluation)

2024-11-25 20:45:05,867 - root - INFO - Evaluation metrics for LogisticRegression:
2024-11-25 20:45:05,867 - root - INFO - Evaluation metrics for LogisticRegression:
2024-11-25 20:45:05,867 - root - INFO - Evaluation metrics for LogisticRegression:
2024-11-25 20:45:05,875 - root - INFO - Accuracy: 0.6443670150987224
2024-11-25 20:45:05,875 - root - INFO - Accuracy: 0.6443670150987224
2024-11-25 20:45:05,875 - root - INFO - Accuracy: 0.6443670150987224
2024-11-25 20:45:05,884 - root - INFO - Precision: 0.16945996275605213
2024-11-25 20:45:05,884 - root - INFO - Precision: 0.16945996275605213
2024-11-25 20:45:05,884 - root - INFO - Precision: 0.16945996275605213
2024-11-25 20:45:05,894 - root - INFO - Recall: 0.7083162638263008
2024-11-25 20:45:05,894 - root - INFO - Recall: 0.7083162638263008
2024-11-25 20:45:05,894 - root - INFO - Recall: 0.7083162638263008
2024-11-25 20:45:05,903 - root - INFO - F1 Score: 0.27348940208794686
2024-11-25 20:45:05,903 - root - INFO - F1 Score: 0.27348940

Logistic Regression Evaluation:
{'accuracy': 0.6443670150987224, 'precision': np.float64(0.16945996275605213), 'recall': np.float64(0.7083162638263008), 'f1': np.float64(0.27348940208794686), 'roc_auc': np.float64(0.7594477170742543), 'confusion_matrix': array([[14915,  8474],
       [  712,  1729]])}

Decision Tree Evaluation:
{'accuracy': 0.9034456058846303, 'precision': np.float64(0.49073750436910174), 'recall': np.float64(0.5751741089717329), 'f1': np.float64(0.5296114673708034), 'roc_auc': np.float64(0.7564399340446334), 'confusion_matrix': array([[21932,  1457],
       [ 1037,  1404]])}

Random Forest Evaluation:
{'accuracy': 0.9571815718157182, 'precision': np.float64(0.9992520568436799), 'recall': np.float64(0.5473166734944694), 'f1': np.float64(0.7072525145579672), 'roc_auc': np.float64(0.7720956722391218), 'confusion_matrix': array([[23388,     1],
       [ 1105,  1336]])}

Gradient Boosting Evaluation:
{'accuracy': 0.9571815718157182, 'precision': np.float64(0.99925205684367

In [14]:
# Extract the model objects from the training results for credit card fraud detection
credit_logistic_model = credit_logistic_results["model"]
credit_decision_tree_model = credit_decision_tree_results["model"]
credit_random_forest_model = credit_random_forest_results["model"]
credit_gradient_boosting_model = credit_gradient_boosting_results["model"]

# Evaluate each credit card fraud detection model
credit_logistic_evaluation = evaluate_model(credit_logistic_model, X_test_credit, y_test_credit)
credit_decision_tree_evaluation = evaluate_model(credit_decision_tree_model, X_test_credit, y_test_credit)
credit_random_forest_evaluation = evaluate_model(credit_random_forest_model, X_test_credit, y_test_credit)
credit_gradient_boosting_evaluation = evaluate_model(credit_gradient_boosting_model, X_test_credit, y_test_credit)

# Display evaluation results for credit card fraud detection models
print("Logistic Regression Evaluation (Credit Card Fraud):")
print(credit_logistic_evaluation)

print("\nDecision Tree Evaluation (Credit Card Fraud):")
print(credit_decision_tree_evaluation)

print("\nRandom Forest Evaluation (Credit Card Fraud):")
print(credit_random_forest_evaluation)

print("\nGradient Boosting Evaluation (Credit Card Fraud):")
print(credit_gradient_boosting_evaluation)


2024-11-25 20:45:09,188 - root - INFO - Evaluation metrics for LogisticRegression:
2024-11-25 20:45:09,188 - root - INFO - Evaluation metrics for LogisticRegression:
2024-11-25 20:45:09,188 - root - INFO - Evaluation metrics for LogisticRegression:
2024-11-25 20:45:09,195 - root - INFO - Accuracy: 0.9776019455115779
2024-11-25 20:45:09,195 - root - INFO - Accuracy: 0.9776019455115779
2024-11-25 20:45:09,195 - root - INFO - Accuracy: 0.9776019455115779
2024-11-25 20:45:09,204 - root - INFO - Precision: 0.05965697240865026
2024-11-25 20:45:09,204 - root - INFO - Precision: 0.05965697240865026
2024-11-25 20:45:09,204 - root - INFO - Precision: 0.05965697240865026
2024-11-25 20:45:09,214 - root - INFO - Recall: 0.8888888888888888
2024-11-25 20:45:09,214 - root - INFO - Recall: 0.8888888888888888
2024-11-25 20:45:09,214 - root - INFO - Recall: 0.8888888888888888
2024-11-25 20:45:09,220 - root - INFO - F1 Score: 0.11180992313067785
2024-11-25 20:45:09,220 - root - INFO - F1 Score: 0.11180992

Logistic Regression Evaluation (Credit Card Fraud):
{'accuracy': 0.9776019455115779, 'precision': np.float64(0.05965697240865026), 'recall': np.float64(0.8888888888888888), 'f1': np.float64(0.11180992313067785), 'roc_auc': np.float64(0.9754564584706141), 'confusion_matrix': array([[55395,  1261],
       [   10,    80]])}

Decision Tree Evaluation (Credit Card Fraud):
{'accuracy': 0.999066013463504, 'precision': np.float64(0.6989247311827957), 'recall': np.float64(0.7222222222222222), 'f1': np.float64(0.7103825136612022), 'roc_auc': np.float64(0.8608640057736359), 'confusion_matrix': array([[56628,    28],
       [   25,    65]])}

Random Forest Evaluation (Credit Card Fraud):
{'accuracy': 0.9995241955380115, 'precision': np.float64(0.9846153846153847), 'recall': np.float64(0.7111111111111111), 'f1': np.float64(0.8258064516129032), 'roc_auc': np.float64(0.9369730380620666), 'confusion_matrix': array([[56655,     1],
       [   26,    64]])}

Gradient Boosting Evaluation (Credit Card Fra

In [15]:
# Explaining Logistic Regression Model
print("Explaining Logistic Regression Model:")

# SHAP explanation for Logistic Regression
logistic_shap_values = explain_model_shap(logistic_model, X_train, X_test, model_type='lr')

# LIME explanation for Logistic Regression
print("LIME explanation for Logistic Regression:")
explanation_lime_lr = explain_model_lime(logistic_model, X_train, X_test, idx=0)


# Explaining Decision Tree Model
print("Explaining Decision Tree Model:")

# SHAP explanation for Decision Tree
decision_tree_shap_values = explain_model_shap(decision_tree_model, X_train, X_test, model_type='rf')

# LIME explanation for Decision Tree
print("LIME explanation for Decision Tree:")
explanation_lime_dt = explain_model_lime(decision_tree_model, X_train, X_test, idx=0)


# Explaining Random Forest Model
print("Explaining Random Forest Model:")

# SHAP explanation for Random Forest
random_forest_shap_values = explain_model_shap(random_forest_model, X_train, X_test, model_type='rf')

# LIME explanation for Random Forest
print("LIME explanation for Random Forest:")
explanation_lime_rf = explain_model_lime(random_forest_model, X_train, X_test, idx=0)


# Explaining Gradient Boosting Model
print("Explaining Gradient Boosting Model:")

# SHAP explanation for Gradient Boosting
gradient_boosting_shap_values = explain_model_shap(gradient_boosting_model, X_train, X_test, model_type='rf')

# LIME explanation for Gradient Boosting
print("LIME explanation for Gradient Boosting:")
explanation_lime_gb = explain_model_lime(gradient_boosting_model, X_train, X_test, idx=0)


2024-11-25 20:45:11,314 - root - INFO - Starting explanation for model type: lr.
2024-11-25 20:45:11,314 - root - INFO - Starting explanation for model type: lr.
2024-11-25 20:45:11,314 - root - INFO - Starting explanation for model type: lr.
2024-11-25 20:45:11,416 - shap - WARNING - Using 103316 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
2024-11-25 20:45:11,416 - shap - WARNING - Using 103316 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
2024-11-25 20:45:11,416 - shap - WARNING - Using 103316 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
2024-11-25 20:45:11,425 - root - INFO - Using KernelExplainer for Logistic Regression model.
2024-11-25 20:45:11,425 - ro

Explaining Logistic Regression Model:


  0%|          | 0/103316 [01:19<?, ?it/s]
2024-11-25 20:46:31,317 - root - ERROR - Error in explaining model: Unable to allocate 25.6 GiB for an array with shape (2076, 1653056) and data type float64
2024-11-25 20:46:31,317 - root - ERROR - Error in explaining model: Unable to allocate 25.6 GiB for an array with shape (2076, 1653056) and data type float64
2024-11-25 20:46:31,317 - root - ERROR - Error in explaining model: Unable to allocate 25.6 GiB for an array with shape (2076, 1653056) and data type float64


MemoryError: Unable to allocate 25.6 GiB for an array with shape (2076, 1653056) and data type float64

In [ ]:
print(X_train.shape)  # Output will be in the form (num_samples, num_features)


In [ ]:
# Explaining Logistic Regression Model (Credit Card Fraud)
print("Explaining Logistic Regression Model (Credit Card Fraud):")

# SHAP explanation for Logistic Regression
credit_logistic_shap_values = explain_model_shap(
    credit_logistic_model, X_train_credit, X_test_credit, model_type='lr'
)

# LIME explanation for Logistic Regression
print("LIME Explanation for Logistic Regression (Credit Card Fraud):")
credit_explanation_lime_lr = explain_model_lime(
    credit_logistic_model, X_train_credit, X_test_credit, idx=0
)


# Explaining Decision Tree Model (Credit Card Fraud)
print("Explaining Decision Tree Model (Credit Card Fraud):")

# SHAP explanation for Decision Tree
credit_decision_tree_shap_values = explain_model_shap(
    credit_decision_tree_model, X_train_credit, X_test_credit, model_type='rf'
)

# LIME explanation for Decision Tree
print("LIME Explanation for Decision Tree (Credit Card Fraud):")
credit_explanation_lime_dt = explain_model_lime(
    credit_decision_tree_model, X_train_credit, X_test_credit, idx=0
)


# Explaining Random Forest Model (Credit Card Fraud)
print("Explaining Random Forest Model (Credit Card Fraud):")

# SHAP explanation for Random Forest
credit_random_forest_shap_values = explain_model_shap(
    credit_random_forest_model, X_train_credit, X_test_credit, model_type='rf'
)

# LIME explanation for Random Forest
print("LIME Explanation for Random Forest (Credit Card Fraud):")
credit_explanation_lime_rf = explain_model_lime(
    credit_random_forest_model, X_train_credit, X_test_credit, idx=0
)


# Explaining Gradient Boosting Model (Credit Card Fraud)
print("Explaining Gradient Boosting Model (Credit Card Fraud):")

# SHAP explanation for Gradient Boosting
credit_gradient_boosting_shap_values = explain_model_shap(
    credit_gradient_boosting_model, X_train_credit, X_test_credit, model_type='rf'
)

# LIME explanation for Gradient Boosting
print("LIME Explanation for Gradient Boosting (Credit Card Fraud):")
credit_explanation_lime_gb = explain_model_lime(
    credit_gradient_boosting_model, X_train_credit, X_test_credit, idx=0
)


In [ ]:

# # Define file paths for saving models
# mlp_model_path = '../models/mlp_model.h5'
# rnn_model_path = '../models/rnn_model.h5'
# cnn_model_path = '../models/cnn_model.h5'
# lstm_model_path = '../models/lstm_model.h5'

# # Train and save the MLP model
# print("Training MLP model...")
# mlp_model, mlp_history = train_mlp(X_train, X_test, y_train, y_test, mlp_model_path)
# print("\nMLP Model Training History:")
# print(mlp_history)



# # Train and save the LSTM model
# print("\nTraining LSTM model...")
# lstm_model, lstm_history = train_lstm(X_train, X_test, y_train, y_test, lstm_model_path)
# print("\nLSTM Model Training History:")
# print(lstm_history)

In [ ]:
# # Train and save the CNN model
# print("\nTraining CNN model...")
# cnn_model, cnn_history = train_cnn(X_train, X_test, y_train, y_test, cnn_model_path)
# print("\nCNN Model Training History:")
# print(cnn_history)

# # Train and save the RNN model
# print("\nTraining RNN model...")
# rnn_model, rnn_history = train_rnn(X_train, X_test, y_train, y_test, rnn_model_path)
# print("\nRNN Model Training History:")
# print(rnn_history)

In [ ]:
# # Evaluate the MLP, RNN, and LSTM models
# mlp_evaluation = evaluate_model(mlp_model, X_test, y_test)
# rnn_evaluation = evaluate_model(rnn_model, X_test, y_test)
# lstm_evaluation = evaluate_model(lstm_model, X_test, y_test)

# # Display evaluation results
# print("\nMLP Model Evaluation:")
# print(mlp_evaluation)

# print("\nRNN Model Evaluation:")
# print(rnn_evaluation)

# print("\nLSTM Model Evaluation:")
# print(lstm_evaluation)


In [ ]:
# # Paths to the saved models
# mlp_model_path = '../models/mlp_model.h5'
# rnn_model_path = '../models/rnn_model.h5'
# cnn_model_path = '../models/cnn_model.h5'
# lstm_model_path = '../models/lstm_model.h5'

# # Load the saved models
# mlp_model = load_model(mlp_model_path)
# rnn_model = load_model(rnn_model_path)
# cnn_model = load_model(cnn_model_path)
# lstm_model = load_model(lstm_model_path)

# # Test the models and evaluate them
# print("Evaluating MLP model...")
# mlp_evaluation = evaluate_model(mlp_model, X_test, y_test)
# print("MLP Model Evaluation:", mlp_evaluation)

# print("\nEvaluating RNN model...")
# rnn_evaluation = evaluate_model(rnn_model, X_test, y_test)
# print("RNN Model Evaluation:", rnn_evaluation)

# print("\nEvaluating CNN model...")
# cnn_evaluation = evaluate_model(cnn_model, X_test, y_test)
# print("CNN Model Evaluation:", cnn_evaluation)

# print("\nEvaluating LSTM model...")
# lstm_evaluation = evaluate_model(lstm_model, X_test, y_test)
# print("LSTM Model Evaluation:", lstm_evaluation)